## Causal Inference

In [28]:
import pandas as pd


In [29]:
df = pd.read_json('/workspaces/chat-gpt-failures/data/causal_prompting_pd_T2.json')

In [30]:
df.head()

,y_levenshtein,t_binary_treatment,t_treatment,z_prompt_size,z_n_ast_errors,z_ast_levels,z_n_whitespaces,z_complexity,z_nloc,z_token_counts,z_n_ast_nodes
0,364,False,control,16,0,9,101,1,12,48,71
1,926,False,control,175,0,17,417,13,100,240,375
2,771,False,control,9,0,13,208,3,17,153,253
3,835,False,control,50,0,12,154,1,14,137,249
4,302,False,control,38,0,13,127,3,21,82,137


## I. Create a causal model from the data and given graph.

In [31]:
from dowhy import CausalModel

In [32]:
df[df["t_treatment"]!="T2"]

,y_levenshtein,t_binary_treatment,t_treatment,z_prompt_size,z_n_ast_errors,z_ast_levels,z_n_whitespaces,z_complexity,z_nloc,z_token_counts,z_n_ast_nodes
0,364,False,control,16,0,9,101,1,12,48,71
1,926,False,control,175,0,17,417,13,100,240,375
2,771,False,control,9,0,13,208,3,17,153,253
3,835,False,control,50,0,12,154,1,14,137,249
4,302,False,control,38,0,13,127,3,21,82,137
...,...,...,...,...,...,...,...,...,...,...,...
5841,654,True,T1,16,0,11,5,1,44,33,51
5842,592,True,T1,30,0,9,140,2,6,31,118
5843,280,True,T1,19,0,9,51,1,7,39,55
5844,504,True,T1,110,0,17,603,6,33,204,328


In [33]:
model_T1 = CausalModel(
        data = df[df["t_treatment"]!="T2"],
        treatment = ['t_binary_treatment'],
        outcome = ['y_levenshtein'],
        effect_modifiers = ['z_ast_levels','z_n_ast_nodes'],
        common_causes= ['z_prompt_size','z_n_whitespaces','z_token_counts','z_nloc']
    )

In [34]:
model_T2 = CausalModel(
        data = df[df["t_treatment"]!="T1"],
        treatment = ['t_binary_treatment'],
        outcome = ['y_levenshtein'],
        #effect_modifiers = ['z_n_ast_errors','z_ast_levels','z_n_whitespaces','z_complexity','z_nloc','z_token_counts','z_n_ast_nodes'],
        effect_modifiers = ['z_ast_levels','z_n_ast_nodes'],
        common_causes= ['z_prompt_size','z_n_whitespaces','z_token_counts','z_nloc']
    )

## II. Identify causal effect and return target estimands.

Find a math expression to identify de-confounders form confounders

In [35]:
identified_estimand_T1 = model_T1.identify_effect()

In [36]:
identified_estimand_T2 = model_T2.identify_effect()

# III. Estimate the target estimand using a statistical method.


In [37]:
causal_estimate_match_T1 = model_T1.estimate_effect(identified_estimand_T1,method_name="backdoor.propensity_score_weighting") #"backdoor.propensity_score_stratification"

propensity_score_weighting


/opt/conda/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.11/site-packages/dowhy/causal_estimators/propensity_score_estimator.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._data[self.propensity_score_column] = self.propensity_score_model.predict_proba(self._observed_common_causes)[:, 1]
/opt/conda/lib/python3.11/site-packages/dowhy/causal_estimators/propensity_score_weighting_estimator.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,c

In [38]:
causal_estimate_match_T2 = model_T2.estimate_effect(identified_estimand_T2,method_name="backdoor.propensity_score_weighting")

propensity_score_weighting


/opt/conda/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.11/site-packages/dowhy/causal_estimators/propensity_score_estimator.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._data[self.propensity_score_column] = self.propensity_score_model.predict_proba(self._observed_common_causes)[:, 1]
/opt/conda/lib/python3.11/site-packages/dowhy/causal_estimators/propensity_score_weighting_estimator.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,c

In [39]:
print(causal_estimate_match_T1)

*** Causal Estimate ***

## Identified estimand
Estimand type: nonparametric-ate

### Estimand : 1
Estimand name: backdoor
Estimand expression:
          d                                                                   
─────────────────────(E[y_levenshtein|z_nloc,z_token_counts,z_prompt_size,z_n_
d[t_binary_treatment]                                                         

             
whitespaces])
             
Estimand assumption 1, Unconfoundedness: If U→{t_binary_treatment} and U→y_levenshtein then P(y_levenshtein|t_binary_treatment,z_nloc,z_token_counts,z_prompt_size,z_n_whitespaces,U) = P(y_levenshtein|t_binary_treatment,z_nloc,z_token_counts,z_prompt_size,z_n_whitespaces)

## Realized estimand
b: y_levenshtein~t_binary_treatment+z_nloc+z_token_counts+z_prompt_size+z_n_whitespaces
Target units: ate

## Estimate
Mean value: 111.05884365378029



In [40]:
print(causal_estimate_match_T2)

*** Causal Estimate ***

## Identified estimand
Estimand type: nonparametric-ate

### Estimand : 1
Estimand name: backdoor
Estimand expression:
          d                                                                   
─────────────────────(E[y_levenshtein|z_nloc,z_token_counts,z_prompt_size,z_n_
d[t_binary_treatment]                                                         

             
whitespaces])
             
Estimand assumption 1, Unconfoundedness: If U→{t_binary_treatment} and U→y_levenshtein then P(y_levenshtein|t_binary_treatment,z_nloc,z_token_counts,z_prompt_size,z_n_whitespaces,U) = P(y_levenshtein|t_binary_treatment,z_nloc,z_token_counts,z_prompt_size,z_n_whitespaces)

## Realized estimand
b: y_levenshtein~t_binary_treatment+z_nloc+z_token_counts+z_prompt_size+z_n_whitespaces
Target units: ate

## Estimate
Mean value: -101.73965104344859



# IV. Refute the obtained estimate using multiple robustness checks.


In [41]:
es_placebo_T1 = model_T1.refute_estimate(
                identified_estimand_T1,
                causal_estimate_match_T1,
                method_name="placebo_treatment_refuter",
                placebo_type="permute"
        )

/opt/conda/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for exam

In [42]:
es_placebo_T2 = model_T2.refute_estimate(
                identified_estimand_T2,
                causal_estimate_match_T2,
                method_name="placebo_treatment_refuter",
                placebo_type="permute"
        )

/opt/conda/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for exam

In [43]:
print(es_placebo_T1)

Refute: Use a Placebo Treatment
Estimated effect:111.05884365378029
New effect:-0.5443276281741123
p value:0.94



In [44]:
print(es_placebo_T2)

Refute: Use a Placebo Treatment
Estimated effect:-101.73965104344859
New effect:-1.3027560114785957
p value:0.94



In [45]:

refute_common_cause_T1 = model_T1.refute_estimate(identified_estimand_T1, causal_estimate_match_T1,
                                       method_name="random_common_cause")

/opt/conda/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for exam

In [46]:
refute_common_cause_T2 = model_T2.refute_estimate(identified_estimand_T2, causal_estimate_match_T2,
                                       method_name="random_common_cause")

/opt/conda/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for exam

In [47]:
print(refute_common_cause_T1)

Refute: Add a random common cause
Estimated effect:111.05884365378029
New effect:111.04830233335804
p value:0.8999999999999999



In [48]:
print(refute_common_cause_T2)

Refute: Add a random common cause
Estimated effect:-101.73965104344859
New effect:-101.74218099588259
p value:0.8600000000000001



In [49]:
res_subset_T1 =model_T1.refute_estimate(identified_estimand_T1, causal_estimate_match_T1,
        method_name="data_subset_refuter", subset_fraction=0.9)


/opt/conda/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for exam

In [50]:
res_subset_T2 =model_T2.refute_estimate(identified_estimand_T2, causal_estimate_match_T2,
        method_name="data_subset_refuter", subset_fraction=0.9)

/opt/conda/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.11/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for exam

In [51]:
print(res_subset_T1)

Refute: Use a subset of data
Estimated effect:111.05884365378029
New effect:111.12923877022368
p value:0.96



In [52]:
print(res_subset_T2)

Refute: Use a subset of data
Estimated effect:-101.73965104344859
New effect:-101.47640803405926
p value:0.92



## Library Test

In [53]:
import dowhy.datasets 

In [54]:
df_T1.shape

NameError: name 'df_T1' is not defined

In [ ]:
data = dowhy.datasets.linear_dataset(beta=10,
        num_common_causes=5, 
        num_instruments = 0,
        num_treatments=1,
        num_samples=5574,
        treatment_is_binary=True,
        outcome_is_binary=False)
df = data["df"]
df

In [ ]:
causal_model = CausalModel(
        data = df,
        treatment = ['v0'],
        outcome = ['y'],
        common_causes = ['W0','W1','W2','W3','W4']
        #common_causes= ['z_nloc']
    )

In [ ]:
identified_estimand = causal_model.identify_effect()
estimate = causal_model.estimate_effect(
        identified_estimand,
        method_name="backdoor.propensity_score_matching")

In [ ]:
print(estimate)

In [ ]:
es_placebo = causal_model.refute_estimate(
                identified_estimand,
                estimate,
                method_name="placebo_treatment_refuter",
                placebo_type="permute"
        )

In [ ]:
print(es_placebo)